In [1]:
import numpy as np
import pandas as pd
import goodreads_api_client as gr
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import requests as req
import requests

from urllib.request import urlopen
import re

# import time
import sys

import sys, os
sys.path.append('..')
import src.helper_functions as hf
import src.scraping as scr

from importlib import reload  
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By

### Getting books and reviews directly from goodreads.

In [3]:
hf = reload(hf)
scr = reload(scr)

In [4]:
popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science'

In [5]:
# Download of book links.
book_links_df = pd.read_csv('../data/big_dataset/book_links_goodreads.csv')
book_links = list(book_links_df['0'])
book_links_copy = book_links.copy()

# removing author links from book_links list.
for book_link in book_links.copy():
    if 'author' in book_link:
        book_links_copy.remove(book_link)

book_links = book_links_copy.copy()

print(f'Total number of books to web-scrap: {len(book_links)}')

Total number of books to web-scrap: 1250


# Get information about the books

In [6]:
# Open the web browser and log in to goodreads:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)
driver = scr.gr_sign_in(driver)

In [9]:
# Script to scrap the books

# Initializing the lists:
books_arr = []
failed_books_list = []

for n in range(0, 1250, 50):
    chunk_arr = []
    for i, book_link in enumerate(book_links[n:n+50]):
        sys.stdout.write(f'book n : {i}\n')
        try:
            #Getting the soup:
            soup, driver = scr.get_the_soup(book_link, driver)
            dct = scr.get_book_info(book_link, soup)
            isbn = dct['isbn']
            reviews, driver = scr.get_reviews_from_book_link(isbn, book_link, driver)
            chunk_arr.append(dct)
        except:
            failed_books_list.append(book_link)
            print('!!!!!!!!!!!!!!!!!!! BOOK FAILED !!!!!!!!!!!!!!!!!!')
    df = pd.DataFrame(chunk_arr)
    df.to_csv(f'../data/gr_books_df_{n}.csv')
    books_arr.extend(chunk_arr)

all_books_df = pd.DataFrame(books_arr)
all_books_df.to_csv(f'../data/all_books_df.csv') 
failed_books_df = pd.DataFrame(failed_books_list)
failed_books_df.to_csv('../data/failed_books_list.csv')